In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
list_book_urls=[]
data_list=[]
"""
In this site contains of multiple paged around 876 pages, each pach contains of nearly 50 books, So it takes a lot of time to 
run all of these pages manually, Hence I have tested for 50 
"""
for index in range(1,877):
    books_page_url = f'https://www.booktopia.com.au/books/new-releases/l2-p{index}.html'
    main_page_response = requests.get(books_page_url)
    main_page_soup = BeautifulSoup(main_page_response.text,"lxml")
    book_urls = [ele.get('href') for ele in  main_page_soup.find_all('a',class_="Product_product-image__1TCds")]
    list_book_urls.extend(book_urls)
def book_data(book_url):
    book_data= dict.fromkeys(('Title_of_the_book','author_name','discount_price','original_price','ISBN-10','Published','Format','Number of Pages','Publisher'),'None')
    page_response = requests.get(book_url)
    page_soup = BeautifulSoup(page_response.text,"lxml")
    try:
        Title_of_the_book = page_soup.find('h1',class_="MuiTypography-root MuiTypography-h1 mui-style-1ngtbwk").text
        book_data.update({'Title_of_the_book':Title_of_the_book})
    except Exception as e:
        print(f'Error:{e},{book_url}')
    try:
        author_name = page_soup.find('span',class_="MuiTypography-root MuiTypography-body1 mui-style-1plnxgp").text
        book_data.update({'author_name':author_name})
    except Exception as e:
        print(f'Error:{e},{book_url}')
    try:    
        discount_price = page_soup.find('p', class_="MuiTypography-root MuiTypography-body1 BuyBox_sale-price__PWbkg mui-style-tgrox").text
        book_data.update({'discount_price':discount_price})
    except Exception as e:
        print(f'Error:{e},{book_url}')
    try:
        original_price = page_soup.find('span', class_="strike").text
        book_data.update({'original_price':original_price})
    except Exception as e:
        print(f'Error:{e},{book_url}')    
    try:
        compare = ['ISBN-10','Published','Format','Number of Pages','Publisher']
        table_data = [ele.text for ele in page_soup.find("div", class_ = "MuiBox-root mui-style-h3npb").find_all("p", class_ = "MuiTypography-root MuiTypography-body1 mui-style-tgrox")]
        for ele in table_data:
            if ele.split(':')[0] in compare:
                book_data.update({ele.split(':')[0]:ele.split(':')[1]})
        return book_data
    except Exception as e:
        print(f'Error at table_data:{e},{book_url}')
        
df=pd.DataFrame(columns=('Title_of_the_book','author_name','discount_price','original_price','ISBN-10','Published','Format','Number of Pages','Publisher'))
with ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(book_data,book_url) for book_url in list_book_urls]
    for future in futures:
        book_info=future.result()
        df_to_append = pd.DataFrame([book_info])
        if not df_to_append.empty:
            df = pd.concat([df, df_to_append], ignore_index=True)

df.to_csv('Books_Details.csv') 
print('Data loaded to csv succesfully')            
print(df )       


Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/kobo-clara-sleepcover-kobo/book/681495009503.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/kobo-clara-sleepcover-kobo/book/681495009510.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/kobo-clara-sleepcover-kobo/book/681495009497.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/kobo-clara-sleepcover-kobo/book/681495009527.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/kobo-clara-bw-ereader-kobo/book/681495009558.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/kobo-clara-clear-ereader-case-kobo/book/681495009596.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/simple-slow-cooker-the-australian-women-s-weekly/book/9781761220593.html
Error:'NoneType' object has no attribute 'text',https://www.booktopia.com.au/sim